In [1]:
import openai
import synthTL

In [2]:
##set openai key
openai.api_key = open("oai_key.txt","r").read()

In [3]:
##set LLM to use
synthTL._LLM_NAME_="gpt-4-0125-preview"

In [4]:
##define the natural language to be translated to LTL
test_str = "GRANT high implies REQUEST is high and then REQUEST is low"

In [5]:
##define the DUT as a LTL formula (optional, can specify None)
##NOTE: this proof-of-concept implementation of synthTL represents DUTs as LTL formulas
formula_DUT = "G (_grant_ -> (_request_ & X !_request_))"
#formula_DUT = None

In [6]:
##define the variable's of the DUT
##NOTE: the variables are interpreted as atomic propositions
synthTL.cur_DUT_variables = ['_grant_','_request_']

In [7]:
##specify number of times to query the LLM for each generated node
num_llm_query = 3

In [8]:
is_hold, metadata = synthTL.generate_validate_and_search(
    test_str,
    formula_DUT,
    num_llm_query,
    )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:58<00:00,  7.26s/it]


new decomposition found:
original text:
GRANT high implies REQUEST is high and then REQUEST is low

abstracted natural language:
_SYMBOL0_ implies _SYMBOL1_ and _SYMBOL2_

num dcmp nodes: 3


is this a desired and valid decomposition? y/n y



new decomposition found:
original text:
GRANT high implies REQUEST is high and then REQUEST is low

abstracted natural language:
_SYMBOL0_ implies _SYMBOL1_ and then _SYMBOL2_

num dcmp nodes: 3


is this a desired and valid decomposition? y/n y


...


new translation found:
abstracted natural language:
_SYMBOL0_ and then _SYMBOL1_

translation:
((_SYMBOL0_) U (_SYMBOL1_))



is this a correct translation? y/n y



new translation found:
abstracted natural language:
_SYMBOL0_ and then _SYMBOL1_

translation:
((_SYMBOL0_) && X (_SYMBOL1_))



is this a correct translation? y/n y



found a specification that holds on the DUT!


In [9]:
if is_hold:
    print("The following is a specification that holds on the DUT")
    holds_graph_list = metadata
    print(synthTL.graph_to_str(holds_graph_list[0]))
else:
    print("did not find a specification that holds on the DUT")
    print("The following are possible culprits:")
    culprit_per_graph_list = metadata
    for i in range(len(culprit_per_graph_list)):
        print("graph #",i)
        for node in culprit_per_graph_list[i]:
            print("\t",synthTL.get_abstract_node_id(node))
            print("\t",synthTL.get_node_translation(node))
            print()
        print()

The following is a specification that holds on the DUT

 Text: SYMBOL0 implies SYMBOL1 and then SYMBOL2
 Translation: G ((_SYMBOL0_) -> ((_SYMBOL1_) && X (_SYMBOL2_)))
	 Text: GRANT high
	 Symbol: SYMBOL0
	 Translation: (_grant_)
	 Text: REQUEST is high
	 Symbol: SYMBOL1
	 Translation: (_request_)
	 Text: REQUEST is low
	 Symbol: SYMBOL2
	 Translation: (!(_request_))
